# Simple Loop - Binary Search with Agent Judge

## Overview
This notebook demonstrates how to build **iterative workflows with loops** using the Agent Framework. You'll learn how to:
- Create feedback loops in workflows with agent steps
- Implement binary search logic using workflow executors
- Use enums for state management and flow control
- Integrate AI agents as judges in decision loops
- Track iterations and workflow completion

## What This Sample Does
The workflow implements a **number guessing game** using binary search:
1. **GuessNumberExecutor** - Performs binary search to guess a target number (1-100)
2. **SubmitToJudgeAgent** - Sends guesses to an AI agent judge
3. **Judge Agent (LLM)** - Responds with ABOVE/BELOW/MATCHED based on the guess
4. **ParseJudgeResponse** - Converts agent response into `NumberSignal` enum
5. **Loop** - Continues until the correct number is guessed

**Target:** `30` (hardcoded)  
**Range:** `1-100`  
**Algorithm:** Binary search  
**Expected Iterations:** ~7 (log₂(100))

## Key Concepts
- **Cyclic Edges:** `add_edge(parse_judge, guess_number)` creates a loop
- **State Management:** Executors maintain state (bounds) across loop iterations
- **Agent Integration:** LLM judges guesses based on target number
- **Enum-Based Flow Control:** `NumberSignal` enum drives decision logic
- **Loop Termination:** Workflow completes when `MATCHED` signal is processed

## Step 1: Import Required Libraries

In [2]:
# Copyright (c) Microsoft. All rights reserved.

import asyncio
from enum import Enum
import os
from dotenv import load_dotenv

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    ExecutorCompletedEvent,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,
    handler,
)
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

# Load environment variables from .env file
load_dotenv('../../.env')

True

## Step 2: Define the NumberSignal Enum
This enum represents the possible states in the guessing workflow.

**States:**
- `INIT` - Start the guessing process
- `ABOVE` - Target is higher than the guess
- `BELOW` - Target is lower than the guess
- `MATCHED` - Guess is correct (loop termination)

In [3]:
class NumberSignal(Enum):
    """Enum to represent number signals for the workflow."""

    # The target number is above the guess.
    ABOVE = "above"
    # The target number is below the guess.
    BELOW = "below"
    # The guess matches the target number.
    MATCHED = "matched"
    # Initial signal to start the guessing process.
    INIT = "init"

## Step 3: Create the GuessNumberExecutor
This executor implements binary search logic with state management.

**State Variables:**
- `_lower`, `_upper` - Current search bounds
- `_guess` - Current guess value

**Logic:**
- `INIT` → Make first guess (midpoint)
- `MATCHED` → Yield final output (success)
- `ABOVE` → Update lower bound, guess higher
- `BELOW` → Update upper bound, guess lower

In [4]:
class GuessNumberExecutor(Executor):
    """An executor that guesses a number."""

    def __init__(self, bound: tuple[int, int], id: str | None = None):
        """Initialize the executor with a target number."""
        super().__init__(id=id or "guess_number")
        self._lower = bound[0]
        self._upper = bound[1]

    @handler
    async def guess_number(self, feedback: NumberSignal, ctx: WorkflowContext[int, str]) -> None:
        """Execute the task by guessing a number."""
        if feedback == NumberSignal.INIT:
            self._guess = (self._lower + self._upper) // 2
            await ctx.send_message(self._guess)
        elif feedback == NumberSignal.MATCHED:
            # The previous guess was correct.
            await ctx.yield_output(f"Guessed the number: {self._guess}")
        elif feedback == NumberSignal.ABOVE:
            # The previous guess was too low.
            # Update the lower bound to the previous guess.
            # Generate a new number that is between the new bounds.
            self._lower = self._guess + 1
            self._guess = (self._lower + self._upper) // 2
            await ctx.send_message(self._guess)
        else:
            # The previous guess was too high.
            # Update the upper bound to the previous guess.
            # Generate a new number that is between the new bounds.
            self._upper = self._guess - 1
            self._guess = (self._lower + self._upper) // 2
            await ctx.send_message(self._guess)

## Step 4: Create the SubmitToJudgeAgent Executor
This executor sends guesses to the AI judge agent.

**Key Points:**
- Constructs prompts with target and guess
- Uses `target_id` to route message to specific agent
- Agent is instructed to respond with one token: MATCHED/ABOVE/BELOW

In [5]:
class SubmitToJudgeAgent(Executor):
    """Send the numeric guess to a judge agent which replies ABOVE/BELOW/MATCHED."""

    def __init__(self, judge_agent_id: str, target: int, id: str | None = None):
        super().__init__(id=id or "submit_to_judge")
        self._judge_agent_id = judge_agent_id
        self._target = target

    @handler
    async def submit(self, guess: int, ctx: WorkflowContext[AgentExecutorRequest]) -> None:
        prompt = (
            "You are a number judge. Given a target number and a guess, reply with exactly one token:"
            " 'MATCHED' if guess == target, 'ABOVE' if the target is above the guess,"
            " or 'BELOW' if the target is below.\n"
            f"Target: {self._target}\nGuess: {guess}\nResponse:"
        )
        await ctx.send_message(
            AgentExecutorRequest(messages=[ChatMessage(Role.USER, text=prompt)], should_respond=True),
            target_id=self._judge_agent_id,
        )

## Step 5: Create the ParseJudgeResponse Executor
This executor converts the LLM's text response into a `NumberSignal` enum.

**Parsing Logic:**
- Extracts text from `AgentExecutorResponse`
- Checks for keywords: MATCHED, ABOVE, BELOW
- Converts to appropriate enum value

In [6]:
class ParseJudgeResponse(Executor):
    """Parse AgentExecutorResponse into NumberSignal for the loop."""

    @handler
    async def parse(self, response: AgentExecutorResponse, ctx: WorkflowContext[NumberSignal]) -> None:
        text = response.agent_run_response.text.strip().upper()
        if "MATCHED" in text:
            await ctx.send_message(NumberSignal.MATCHED)
        elif "ABOVE" in text and "BELOW" not in text:
            await ctx.send_message(NumberSignal.ABOVE)
        else:
            await ctx.send_message(NumberSignal.BELOW)

## Step 6: Build the Workflow with Loop
Create the workflow graph with a cyclic edge to form the loop.

**Workflow Structure:**
```
       ┌─────────────────────────────────────┐
       │                                     │
       ▼                                     │
GuessNumber → SubmitToJudge → JudgeAgent → ParseJudge
     │                                       │
     │                                       │
     └───────── (loop back) ─────────────────┘
```

**Loop Mechanics:**
- `add_edge(parse_judge, guess_number_executor)` creates the loop
- Each iteration: guess → judge → parse → guess (repeat)
- Loop terminates when `MATCHED` signal is processed

In [7]:
async def run_workflow():
    """Main function to run the workflow."""
    # Step 1: Create the executors.
    guess_number_executor = GuessNumberExecutor((1, 100))

    # Agent judge setup
    endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
    chat_client = AzureOpenAIChatClient(
        deployment_name=deployment_name,
        endpoint=endpoint,
        credential=AzureCliCredential()
    )
    judge_agent = AgentExecutor(
        chat_client.create_agent(
            instructions=(
                "You strictly respond with one of: MATCHED, ABOVE, BELOW based on the given target and guess."
            )
        ),
        id="judge_agent",
    )
    submit_to_judge = SubmitToJudgeAgent(judge_agent_id=judge_agent.id, target=30, id="submit_judge")
    parse_judge = ParseJudgeResponse(id="parse_judge")

    # Step 2: Build the workflow with the defined edges.
    # This time we are creating a loop in the workflow.
    workflow = (
        WorkflowBuilder()
        .add_edge(guess_number_executor, submit_to_judge)
        .add_edge(submit_to_judge, judge_agent)
        .add_edge(judge_agent, parse_judge)
        .add_edge(parse_judge, guess_number_executor)  # Loop back!
        .set_start_executor(guess_number_executor)
        .build()
    )

    # Step 3: Run the workflow and print the events.
    print("=== Binary Search Number Guessing Game ===")
    print("Target: 30")
    print("Range: 1-100\n")
    
    iterations = 0
    async for event in workflow.run_stream(NumberSignal.INIT):
        if isinstance(event, ExecutorCompletedEvent) and event.executor_id == guess_number_executor.id:
            iterations += 1
        elif isinstance(event, WorkflowOutputEvent):
            print(f"\n=== Result ===")
            print(f"Final result: {event.data}")
        print(f"{event}")

    # This is essentially a binary search, so the number of iterations should be logarithmic.
    # The maximum number of iterations is [log2(range size)]. For a range of 1 to 100, this is log2(100) which is 7.
    # Subtract because the last round is the MATCHED event.
    print(f"\nGuessed {iterations - 1} times.")
    print(f"Expected: ~{len(bin(100)) - 2} times (log₂(100))")

## Step 7: Execute the Workflow
**Note:** This requires Azure OpenAI access. Make sure you're authenticated via `az login`.

In [8]:
# Uncomment to run (requires Azure OpenAI credentials)
# await run_workflow()

## Expected Output (Sample)
```
=== Binary Search Number Guessing Game ===
Target: 30
Range: 1-100

ExecutorInvokedEvent(executor_id='guess_number')
ExecutorCompletedEvent(executor_id='guess_number')
ExecutorInvokedEvent(executor_id='submit_judge')
ExecutorCompletedEvent(executor_id='submit_judge')
ExecutorInvokedEvent(executor_id='judge_agent')
ExecutorCompletedEvent(executor_id='judge_agent')
ExecutorInvokedEvent(executor_id='parse_judge')
ExecutorCompletedEvent(executor_id='parse_judge')
ExecutorInvokedEvent(executor_id='guess_number')
ExecutorCompletedEvent(executor_id='guess_number')
...

=== Result ===
Final result: Guessed the number: 30
WorkflowOutputEvent(data='Guessed the number: 30')

Guessed 6 times.
Expected: ~7 times (log₂(100))
```

## Key Takeaways

### Loop Creation
✅ **Cyclic Edges:** `add_edge(last, first)` creates a feedback loop  
✅ **Stateful Executors:** Executors maintain state (bounds) across iterations  
✅ **Termination Conditions:** Loops end when no more messages are sent  
✅ **Iteration Tracking:** Count specific executor completions to measure iterations

### Binary Search Implementation
1. **Initialization:** Start with full range (1-100), guess midpoint (50)
2. **Feedback:** Judge responds ABOVE/BELOW/MATCHED
3. **Update Bounds:** Narrow search space based on feedback
4. **New Guess:** Calculate midpoint of updated bounds
5. **Repeat:** Continue until MATCHED signal
6. **Complexity:** O(log n) - guaranteed to find any number in ~7 iterations

### Agent Integration in Loops
- **AI as Validator:** LLM judges guesses without hardcoded logic
- **Dynamic Feedback:** Agent provides natural language reasoning
- **Reliable Parsing:** Extract structured signals from text responses
- **Scalability:** Same pattern works for complex decision-making

### Event Streaming in Loops
- Track each iteration's executor invocations
- Count loop cycles by monitoring specific executor completions
- Observe when the workflow becomes idle (loop termination)

## Loop Patterns

### While-Loop Pattern
```python
# Executor maintains condition state
@handler
async def check_condition(state, ctx):
    if condition_met(state):
        await ctx.yield_output(result)
    else:
        await ctx.send_message(updated_state)

# Loop: check → process → check (repeat)
workflow.add_edge(process, check).add_edge(check, process)
```

### For-Loop Pattern
```python
# Executor maintains counter
@handler
async def iterate(state, ctx):
    if state.counter < max_iterations:
        await ctx.send_message(state.increment())
    else:
        await ctx.yield_output(state.result)
```

### Retry Pattern
```python
# Loop until success or max retries
@handler
async def retry_executor(attempt, ctx):
    if attempt.success or attempt.count > max_retries:
        await ctx.yield_output(attempt)
    else:
        await ctx.send_message(attempt.retry())
```

## Best Practices for Loops
- **Always have termination conditions** to prevent infinite loops
- **Use enums** for clear state transitions
- **Maintain state in executor instances** for complex loop logic
- **Track iteration counts** for debugging and monitoring
- **Test with small ranges** before scaling to large iterations

## Next Steps
- Explore `edge_condition.ipynb` for conditional branching (if/else)
- See `switch_case_edge_group.ipynb` for multi-way branching
- Check `multi_selection_edge_group.ipynb` for parallel execution patterns